# How to solve pressure poisson equation

We need to solve 

$$\mathbf{M}\mathbf{V}=\mathbf{M}\mathbf{G}\mathbf{p}$$

for the pressure.
Note that $\mathbf{M}\mathbf{G}=\mathbf{M}^T\mathbf{M} = \mathbf{A}$ is circulant and symmetric. Therefore, its eigenvectors are the discrete fourier modes (due to the circulant property) with real eigenvalues $\boldsymbol{\lambda}$ (due to the symmetry). By writing $\mathbf{M}\mathbf{V} = \mathbf{r}$ we find the pressure by solving 

$$\mathbf{A}\mathbf{p} = \mathbf{r}.$$

Using the eigendecomposition we can write $\mathbf{A}= \mathbf{F}^{-1}\text{diag}(\boldsymbol{\lambda})\mathbf{F}$, where $\mathbf{F}$ is a discrete fourier transform operator and $\mathbf{F}^{-1}=\mathbf{F}^T$ its inverse. We obtain the eigenvalues by doing

$$ \mathbf{F}\mathbf{A}\mathbf{F}^{-1}\mathbf{1} = \underbrace{\mathbf{F}\mathbf{F}^{-1}}_{=\mathbf{I}}\text{diag}(\boldsymbol{\lambda})\underbrace{\mathbf{F}\mathbf{F}^{-1}}_{=\mathbf{I}}\mathbf{1} = \boldsymbol{\lambda}.$$

After obtaining $\boldsymbol{\lambda}$ this allows us to invert $\mathbf{A}$ and obtain $\mathbf{p}$:

$$\mathbf{A}\mathbf{p} = \mathbf{r} \rightarrow   \mathbf{F}^{-1}\text{diag}(\boldsymbol{\lambda})\mathbf{F}\mathbf{p} = \mathbf{r} \rightarrow \mathbf{p} =  \mathbf{F}^{-1}\text{diag}(\boldsymbol{\lambda})^{-1}\mathbf{F}\mathbf{r}.$$

We can use a fast fourier transform to replace $\mathbf{F}$.

In [1]:
include("source.jl")
include("NS_FVM_solver.jl")


In [2]:
x= collect(LinRange(-pi,pi,129))
y = collect(LinRange(-pi,pi,129))
#z = collect(LinRange(-pi,pi,257))

UPC = 2
fine_mesh = gen_mesh(x,y,UPC = UPC)

# number of unknowns per cell

J = (16,16) # compression in each direction
coarse_mesh = generate_coarse_from_fine_mesh(fine_mesh,J)
MP = gen_mesh_pair(fine_mesh,coarse_mesh)
0



LoadError: UndefVarError: `gen_mesh` not defined

In [3]:
struct setup_struct
    O
    PS
    GS
    mesh
end
    
function gen_setup(mesh)
    O = gen_operators_uniform(mesh)
    PS = gen_pressure_solver(mesh,O)
    GS = gen_grid_swapper(mesh)
    
    return setup_struct(O,PS,GS,mesh)
end

gen_setup (generic function with 1 method)

In [4]:
setup = gen_setup(fine_mesh)
0

LoadError: UndefVarError: `fine_mesh` not defined

# Define RHS of kolmogorov flow

In [5]:
forcing(x) = sin.(4*x[2])

F = fine_mesh.eval_function(forcing)
F = cat(F,zeros(size(F)),dims = fine_mesh.dims + 1)



function gen_kolmogorov_flow(setup,F;Re = 1000)
    
    
    function rhs(V,mesh,t;Re = Re,O=setup.O,PS = setup.PS,F = padding(F,(1,1),circular = true),solve_pressure = true,other_arguments = 0)

        dims = mesh.dims

        pad_V = padding(V,Tuple((3 for i in 1:dims)),circular = true)



        CV = O.C(pad_V)


        DV = (1/Re) .* O.D(pad_V[[(2:end-1) for i in 1:dims]...,:,:])

        rhs = -CV + DV

        #### kolmogorov flow ###
        
        rhs += F .- 0.1*pad_V[[(3:end-2) for i in 1:dims]...,:,:]

        ########################
       
        if solve_pressure 
            r = O.M(rhs)
            p = PS(r)
            return rhs[[(2:end-1) for i in 1:dims]...,:,:] - O.G(padding(p,Tuple((1 for i in 1:dims)),circular = true))
        else
            return rhs[[(2:end-1) for i in 1:dims]...,:,:] 
        end
        
    end
        
    return rhs
end

LoadError: UndefVarError: `fine_mesh` not defined

# Generate divergence free initial condition

In [6]:
max_k = 10
energy_norm = 1
number_of_simulations = 1

setup = gen_setup(fine_mesh)
KF_rhs = gen_kolmogorov_flow(setup,F)

V = generate_random_field(fine_mesh.N,max_k,norm = energy_norm,samples = (fine_mesh.UPC,number_of_simulations))


    
MV = setup.O.M(padding(V,(1,1),circular = true))



p = setup.PS(MV)

Gp = setup.O.G(padding(p,(1,1),circular =true))

V0 = V-Gp

heatmap(setup.O.M(V0)[:,:,1,1])


LoadError: UndefVarError: `fine_mesh` not defined

# Warm-up

In [7]:
t_start = 0
t_end = 10
dt = 0.02
save_every = 50
pre_allocate = true


t_data,w_up_sim_data = simulate(V0,fine_mesh,dt,t_start,t_end,KF_rhs,time_step,save_every = save_every,pre_allocate = pre_allocate) 
0


LoadError: UndefVarError: `simulate` not defined

In [8]:
w_up_sim_data

LoadError: UndefVarError: `w_up_sim_data` not defined

In [9]:
Plots.plot(fine_mesh.ip(w_up_sim_data,w_up_sim_data)[1:end])

LoadError: UndefVarError: `Plots` not defined

# True sim

In [10]:
t_start = 0
t_end = 50
dt = 0.02
save_every = 5
pre_allocate = true


t_data,sim_data = simulate(w_up_sim_data[:,:,:,:,end],fine_mesh,dt,t_start,t_end,KF_rhs,time_step,save_every = save_every,pre_allocate = pre_allocate) 
0
data = reshape(sim_data,(size(sim_data)[1:end-2]...,size(sim_data)[end]*size(sim_data)[end-1]))


LoadError: UndefVarError: `w_up_sim_data` not defined

In [11]:
Plots.plot(fine_mesh.ip(sim_data[:,:,:,1,:],sim_data[:,:,:,1,:])[1:end])

LoadError: UndefVarError: `Plots` not defined

In [12]:
to_plot = setup.O.w(sim_data[:,:,:,1,:])[:,:,1,:]

ymin = minimum(to_plot)
ymax = maximum(to_plot)


anim = @animate for index in 1:size(to_plot)[3]
    Plots.heatmap(to_plot[:,:,index]',color = :brg,clim = (ymin,ymax),yflip=false,xtickfontsize = 10,ytickfontsize = 10
    ,aspect_ratio = :equal,titlefontsize=16,yguidefontsize=18,xguidefontsize=18,title = "Simplified ocean model",right_margin = 35Plots.mm)
    Plots.xlabel!(L"x")
    Plots.ylabel!(L"y")
    #title!(L"t = " * string(round(example_t[index],digits = 1)))
end

gif(anim, "DNS_flow.gif", fps = 30)

LoadError: UndefVarError: `setup` not defined

# Generate local POD model

In [13]:
conserve_momentum = true
### momentum conserving correction of data set

data_hat = sqrt.(MP.omega_tilde) .* setup.GS.A_s_c(data)
reshape_for_local_SVD(data_hat,MP)

POD_modes,S = carry_out_local_SVD(data_hat,MP,subtract_average = conserve_momentum)

0

LoadError: UndefVarError: `MP` not defined

In [14]:
Plots.plot(S,marker = true,yscale = :log)
hline!([0.01])

LoadError: UndefVarError: `Plots` not defined

In [15]:
r =30 #how many modes to include, including u_bar
uniform = true

if conserve_momentum 
    if r >= UPC + 1
        global_POD_modes = local_to_global_modes(POD_modes[[(:) for i in 1:fine_mesh.dims]...,:,1:(r-UPC)],MP)

    else
        global_POD_modes = 0
    end

    global_POD_modes = add_filter_to_modes(global_POD_modes,MP,orthogonalize = !uniform)

else
    global_POD_modes = local_to_global_modes(POD_modes[[(:) for i in 1:fine_mesh.dims]...,:,1:r],MP)
end

PO = gen_projection_operators(global_POD_modes,MP,uniform =uniform)



LoadError: UndefVarError: `fine_mesh` not defined

In [16]:
function true_W(u,setup,setup_bar,PO)
    
    UPC = setup.mesh.UPC
    dims = setup.mesh.dims

    
    a = PO.W(setup.GS.A_s_c(u))
    
    u_bar_c = a[[(:) for i in 1:dims]...,1:UPC,:]

    u_bar_s = setup_bar.GS.A_c_s(u_bar_c)
    
    r = setup_bar.O.M(padding(u_bar_s,Tuple((1 for i in 1:dims)),circular = true))
    p = setup_bar.PS(r)
    
    Gp = setup_bar.O.G(padding(p,Tuple((1 for i in 1:dims)),circular = true))
    
    u_bar_s -= Gp
    
    return cat(u_bar_s,a[[(:) for i in 1:dims]...,UPC+1:end,:],dims = dims+1)
end

function true_R(a,setup,setup_bar,PO)
    
    UPC = setup.mesh.UPC
    dims = setup.mesh.dims
    
    u_bar_s = a[[(:) for i in 1:dims]...,1:UPC,:]
    
    u_bar_s = setup_bar.GS.A_s_c(u_bar_s)
    
    u_r = setup.GS.A_c_s(PO.R(cat(u_bar_s,a[[(:) for i in 1:dims]...,UPC+1:end,:],dims = dims+1)))
    
    r = setup.O.M(padding(u_r,Tuple((1 for i in 1:dims)),circular = true))
    p = setup.PS(r)

    Gp = setup.O.G(padding(p,Tuple((1 for i in 1:dims)),circular = true))
    
    u_r -= Gp
    
    
    return u_r
end


true_R (generic function with 1 method)

# True filter

In [17]:

setup_bar = gen_setup(coarse_mesh)

W(u,setup = setup,setup_bar = setup_bar,PO = PO) = true_W(u,setup,setup_bar,PO)
R(a,setup = setup,setup_bar = setup_bar,PO = PO) = true_R(a,setup,setup_bar,PO)

ref_data = W(data)
#a_rhs = true_W(KF_rhs(data,setup.mesh,0),setup,setup_bar,PO)
F_a = W(F)
rhs_bar = gen_kolmogorov_flow(setup_bar,F_a[:,:,1:2,:];Re = 1000)
0



LoadError: UndefVarError: `coarse_mesh` not defined

In [18]:
scatter(fine_mesh.ip(data,data)[1:end],coarse_mesh.ip(ref_data,ref_data)[1:end])

LoadError: UndefVarError: `fine_mesh` not defined

In [19]:
kernel_sizes = [(1,1),(1,1)]
channels = [r+2,30] # r+2 as the first channels are the coarse rhs
strides = [(1,1),(1,1)]
B = (1,1)
boundary_padding = "c" #[["c","c"];;[0,0]]


constrain_energy =true
dissipation = true
conserve_momentum =true


model = gen_skew_NN(kernel_sizes,channels,strides,r,B,boundary_padding = boundary_padding,UPC = coarse_mesh.UPC,constrain_energy = constrain_energy,dissipation = dissipation,conserve_momentum = conserve_momentum)

LoadError: UndefVarError: `coarse_mesh` not defined

In [20]:
function neural_rhs(a,mesh,t,rhs = rhs_bar,setup = setup_bar,model = model,F = F_a,B=B;other_arguments = 0)
    
    
    dims = mesh.dims
    
    u_s = a[:,:,1:2,:]
    
    RHS = rhs(u_s,mesh,t,solve_pressure = false)
    
    RHS_c = setup.GS.A_s_c(RHS)
    u_c = setup.GS.A_s_c(u_s)
    
    a_c = cat(u_c,a[:,:,3:end,:],dims = dims+1)
    
    input = cat(RHS_c,a,dims = dims +1)  

    nn_output = model.eval(input,a = padding(a_c,((2*[B...])...,),circular = true))
    
    nn_output = cat(setup.GS.A_c_s(nn_output[:,:,1:2,:]),nn_output[:,:,3:end,:],dims = dims+1)
    ### find pressure based on NN_output
    #+ nn_output[:,:,1:2,:]
    r = setup.O.M(padding(RHS + nn_output[:,:,1:2,:] ,(1,1),circular = true))
    
    p = setup.PS(r)
    
    Gp = setup.O.G(padding(p,(1,1),circular = true))
     
    ####################################

    physics_rhs = cat( RHS -Gp , - 0.1 * a[:,:,3:end,:] .+ F[:,:,3:end,:] ,dims = dims +1) # from kolmogorov flow

    return nn_output +   physics_rhs
    
    
end

neural_rhs (generic function with 6 methods)

In [21]:
function galerkin_rhs(a,mesh,t,setup = setup,W=W,R = R,rhs = KF_rhs;other_arguments = 0)
    V = R(a)
    dadt = W(rhs(V,setup.mesh,t))
    return dadt
end

galerkin_rhs (generic function with 5 methods)

In [22]:
galerkin_rhs(ref_data,coarse_mesh,0)

LoadError: UndefVarError: `ref_data` not defined

# Trajectory fitting

In [23]:
simulation_indexes = collect(1:size(sim_data)[end-1])'
simulation_indexes = cat([simulation_indexes for i in 1:size(sim_data)[end]]...,dims = fine_mesh.dims + 2)
simulation_indexes = simulation_indexes[1:end]
simulation_times = t_data[1:end]

ref_data_trajectory = reshape(ref_data,(size(ref_data)[1:coarse_mesh.dims+1]...,size(sim_data)[end-1],size(sim_data)[end]))
        

sim_interpolator = gen_time_interpolator(t_data,ref_data_trajectory)





LoadError: UndefVarError: `sim_data` not defined

In [24]:
# remove data that lies at the end of the simulation
buffer_dt = 0.6
select = buffer_dt .< maximum(simulation_times) .- simulation_times

traj_data = ref_data[[(:) for i in 1:coarse_mesh.dims+1]...,select]
traj_indexes = simulation_indexes[select]
traj_times = simulation_times[select]

LoadError: UndefVarError: `simulation_times` not defined

In [25]:
traj_dt = 0.1
traj_steps = 5

function trajectory_fitting_loss(input,indexes,times,dt = traj_dt,steps = traj_steps,sim_interpolator = sim_interpolator,neural_rhs = neural_rhs,coarse_mesh = coarse_mesh)
    dims = length(size(input)) - 2
    
    t_start = reshape(times,([1 for i in 1:dims+1]...,size(times)[1]))
    t_end =  t_start .+ steps *dt
    t,result = simulate(input,coarse_mesh,dt,t_start,t_end,neural_rhs,time_step,save_every = 1,pre_allocate = false) 
    reference = stop_gradient() do
        sim_interpolator(t,simulation_indexes = indexes)
    end
    return Flux.Losses.mse(result,reference)
end    
    
    
    
    

trajectory_fitting_loss (generic function with 6 methods)

In [26]:
batchsize = 5
trajectory_fitting_data_loader = Flux.Data.DataLoader((traj_data,traj_indexes,traj_times), batchsize=batchsize,shuffle=true)

LoadError: UndefVarError: `Flux` not defined

In [27]:
select = rand(collect(1:prod(size(traj_indexes))),(50))
sqrt.(trajectory_fitting_loss(traj_data[:,:,:,select],traj_indexes[select],traj_times[select]))

LoadError: UndefVarError: `traj_indexes` not defined

In [28]:
opt = ADAM()

LoadError: UndefVarError: `ADAM` not defined

In [29]:
ps = Flux.params(model.CNN,model.B_mats...)
epochs = 90
losses = zeros(epochs)
for epoch in tqdm(1:epochs)
    Flux.train!(trajectory_fitting_loss,ps, trajectory_fitting_data_loader, opt)
    train_loss = trajectory_fitting_loss(traj_data[:,:,:,select],traj_indexes[select],traj_times[select])
    losses[epoch] = train_loss
end

LoadError: UndefVarError: `Flux` not defined

In [30]:
plot(sqrt.(losses))

LoadError: UndefVarError: `losses` not defined

In [31]:
t_start = 0
t_end = 20
dt = 0.1
save_every = 1
pre_allocate = true
init_cond = true_W(w_up_sim_data[:,:,:,1,end:end],setup,setup_bar,PO)

t,pred_sim = simulate(init_cond,coarse_mesh,dt,t_start,t_end,neural_rhs,time_step,save_every = save_every,pre_allocate = pre_allocate) 
t_pred = t[1:end]
pred_sim = pred_sim[:,:,:,1,:]



LoadError: UndefVarError: `w_up_sim_data` not defined

In [32]:
Plots.plot(fine_mesh.ip(sim_data[:,:,:,1,:],sim_data[:,:,:,1,:])[1:end])

LoadError: UndefVarError: `Plots` not defined

In [33]:
plot(coarse_mesh.ip(pred_sim,pred_sim)[1:end])
plot!(coarse_mesh.ip(W(sim_data[:,:,:,1,:]),W(sim_data[:,:,:,1,:]))[1:end])

LoadError: UndefVarError: `coarse_mesh` not defined

In [34]:
to_plot = setup.O.w(R(pred_sim))[:,:,1,:]
#to_plot = setup_bar.O.w(pred_sim[:,:,1:2,:])[:,:,1,:]


ymin = minimum(to_plot)
ymax = maximum(to_plot)


anim = @animate for index in 1:size(to_plot)[3]
    Plots.heatmap(to_plot[:,:,index]',color = :brg,clim = (ymin,ymax),yflip=false,xtickfontsize = 10,ytickfontsize = 10
    ,aspect_ratio = :equal,titlefontsize=16,yguidefontsize=18,xguidefontsize=18,title = "Simplified ocean model",right_margin = 35Plots.mm)
    Plots.xlabel!(L"x")
    Plots.ylabel!(L"y")
    #title!(L"t = " * string(round(example_t[index],digits = 1)))
end

gif(anim, "DNS_flow.gif", fps = 30)

LoadError: UndefVarError: `setup` not defined

In [35]:
to_plot = setup.O.w(R(W(sim_data[:,:,:,1,:])))[:,:,1,:]

ymin = minimum(to_plot)
ymax = maximum(to_plot)


anim = @animate for index in 1:size(to_plot)[3]
    Plots.heatmap(to_plot[:,:,index]',color = :brg,clim = (ymin,ymax),yflip=false,xtickfontsize = 10,ytickfontsize = 10
    ,aspect_ratio = :equal,titlefontsize=16,yguidefontsize=18,xguidefontsize=18,title = "Simplified ocean model",right_margin = 35Plots.mm)
    Plots.xlabel!(L"x")
    Plots.ylabel!(L"y")
    #title!(L"t = " * string(round(example_t[index],digits = 1)))
end

gif(anim, "DNS_flow.gif", fps = 30)

LoadError: UndefVarError: `setup` not defined

In [36]:
lit = coarse_mesh.ip(pred_sim,pred_sim,combine_channels = false)[1,1,:,:]

LoadError: UndefVarError: `coarse_mesh` not defined